## 2-3-1. create a subgraph (1) that consist of a single Linear layer of size MxKxN

In [51]:
import onnx
from onnx import helper
import numpy as np

# 定義線性層的形狀參數
M, K, N = 1, 9216, 4096  # 假設批量大小為 1，與 AlexNet 第一個 Linear 層對應

# 定義名稱
input_name = "input"  # 輸入名稱
weight_name = "weight"  # 權重名稱
bias_name = "bias"  # 偏置名稱（可選）
output_name = "output"  # 輸出名稱

# 創建 Gemm 節點（模擬線性層）
gemm_node = helper.make_node(
    "Gemm",  # 運算子類型
    inputs=[input_name, weight_name, bias_name],  # 輸入：數據、權重、偏置
    outputs=[output_name],  # 輸出
    name="linear_layer",  # 節點名稱
    alpha=1.0,  # 縮放因子（默認）
    beta=1.0,  # 偏置縮放因子（默認）
    transA=0,  # 不轉置 A
    transB=1   # 轉置 B
)

# 創建輸入、權重和偏置的形狀資訊
input_info = helper.make_tensor_value_info(input_name, onnx.TensorProto.FLOAT, [M, K])
weight_info = helper.make_tensor_value_info(weight_name, onnx.TensorProto.FLOAT, [N, K])
bias_info = helper.make_tensor_value_info(bias_name, onnx.TensorProto.FLOAT, [N])
output_info = helper.make_tensor_value_info(output_name, onnx.TensorProto.FLOAT, [M, N])

# 創建初始值（Initializer）作為示例數據
# 隨機生成權重和偏置（用於演示）
weight_data = np.random.randn(N, K).astype(np.float32)
bias_data = np.random.randn(N).astype(np.float32)

weight_initializer = helper.make_tensor(
    name=weight_name,
    data_type=onnx.TensorProto.FLOAT,
    dims=weight_data.shape,
    vals=weight_data.flatten().tolist()
)

bias_initializer = helper.make_tensor(
    name=bias_name,
    data_type=onnx.TensorProto.FLOAT,
    dims=bias_data.shape,
    vals=bias_data.flatten().tolist()
)

# 創建圖（Graph），將 initializers 添加到 graph.initializer
graph = helper.make_graph(
    [gemm_node],  # 節點列表
    "LinearSubgraph",  # 圖名稱
    inputs=[input_info],  # 輸入
    outputs=[output_info]  # 輸出
)
graph.initializer.extend([weight_initializer, bias_initializer])  # 將 initializers 添加到 graph

# 創建模型
model = helper.make_model(graph, producer_name="LinearSubgraphExample")

# 驗證模型
onnx.checker.check_model(model)
print("Linear subgraph (1) created successfully.")

# 保存子圖為 ONNX 模型（用於測試）
onnx.save(model, "./linear_subgraph.onnx")

Linear subgraph (1) created successfully.


In [53]:
import onnx
print(f"ONNX version: {onnx.__version__}")
model = onnx.load("./alexnet.onnx")
print(f"Opset version: {model.opset_import[0].version}")

ONNX version: 1.13.1
Opset version: 17


In [1]:
import onnx
from onnx import helper
import numpy as np
import math

# 定義名稱
input_a_name = "input_A"  # 128x128 輸入矩陣 A
input_b_name = "input_B"  # 128x128 輸入矩陣 B
output_c_name = "output_C"  # 128x128 輸出矩陣 C

# 子塊名稱
a_tiles = ["A00", "A01", "A10", "A11"]
b_tiles = ["B00", "B01", "B10", "B11"]
c_tiles = ["C00", "C01", "C10", "C11"]

# 創建 Split 節點（分割 A 和 B，使用 num_outputs 或均勻分割）
# 沿 axis=0 分割 A（2 個輸出）
split_a_row = helper.make_node(
    "Split",
    inputs=[input_a_name],
    outputs=["temp_row0", "temp_row1"],
    axis=0,  # 沿行分割
    # num_outputs=2,  # 僅在較低 Opset 中有效（如 Opset 1-10），在 Opset 17 可能不支援
    name="split_a_row"
)

# 沿 axis=1 分割 A 的每行（每個 2 個輸出）
split_a_col0 = helper.make_node(
    "Split",
    inputs=["temp_row0"],
    outputs=[a_tiles[0], a_tiles[1]],  # A00, A01
    axis=1,  # 沿列分割
    # num_outputs=2,
    name="split_a_col0"
)

split_a_col1 = helper.make_node(
    "Split",
    inputs=["temp_row1"],
    outputs=[a_tiles[2], a_tiles[3]],  # A10, A11
    axis=1,
    # num_outputs=2,
    name="split_a_col1"
)

# 沿 axis=0 分割 B（行），沿 axis=1 分割（列）
split_b_row = helper.make_node(
    "Split",
    inputs=[input_b_name],
    outputs=["temp_b_row0", "temp_b_row1"],
    axis=0,
    # num_outputs=2,
    name="split_b_row"
)

split_b_col0 = helper.make_node(
    "Split",
    inputs=["temp_b_row0"],
    outputs=[b_tiles[0], b_tiles[1]],  # B00, B01
    axis=1,
    # num_outputs=2,
    name="split_b_col0"
)

split_b_col1 = helper.make_node(
    "Split",
    inputs=["temp_b_row1"],
    outputs=[b_tiles[2], b_tiles[3]],  # B10, B11
    axis=1,
    # num_outputs=2,
    name="split_b_col1"
)

# 創建 2DMM（MatMul）和 Sum（Add）節點
matmul_nodes = []
add_nodes = []

for i in range(2):
    for j in range(2):
        # 計算 C_ij 的每個部分
        matmul1_output = f"matmul1_{i}_{j}"
        matmul2_output = f"matmul2_{i}_{j}"
        matmul_nodes.append(helper.make_node(
            "MatMul",
            inputs=[a_tiles[i * 2], b_tiles[j * 2]],  # A_i0 * B_0j
            outputs=[matmul1_output]
        ))
        matmul_nodes.append(helper.make_node(
            "MatMul",
            inputs=[a_tiles[i * 2 + 1], b_tiles[j * 2 + 1]],  # A_i1 * B_1j
            outputs=[matmul2_output]
        ))

        # 累加（Sum）
        sum_output = c_tiles[i * 2 + j]
        add_nodes.append(helper.make_node(
            "Add",
            inputs=[matmul1_output, matmul2_output],
            outputs=[sum_output]
        ))

# 創建 Concat 節點（沿 axis=0 和 axis=1 拼接）
concat_row0 = helper.make_node(
    "Concat",
    inputs=[c_tiles[0], c_tiles[1]],  # C00, C01
    outputs=["temp_concat_row0"],
    axis=1,  # 沿列拼接
    name="concat_row0"
)

concat_row1 = helper.make_node(
    "Concat",
    inputs=[c_tiles[2], c_tiles[3]],  # C10, C11
    outputs=["temp_concat_row1"],
    axis=1,
    name="concat_row1"
)

concat_final = helper.make_node(
    "Concat",
    inputs=["temp_concat_row0", "temp_concat_row1"],
    outputs=[output_c_name],
    axis=0,  # 沿行拼接
    name="concat_final"
)

# 創建圖（Graph）
graph = helper.make_graph(
    [split_a_row, split_a_col0, split_a_col1, split_b_row, split_b_col0, split_b_col1] + matmul_nodes + add_nodes + [concat_row0, concat_row1, concat_final],
    "TiledMatrixMultiplicationSubgraph",  # 圖名稱
    inputs=[
        helper.make_tensor_value_info(input_a_name, onnx.TensorProto.FLOAT, [128, 128]),
        helper.make_tensor_value_info(input_b_name, onnx.TensorProto.FLOAT, [128, 128])
    ],
    outputs=[helper.make_tensor_value_info(output_c_name, onnx.TensorProto.FLOAT, [128, 128])]
)

# 創建模型，指定 Opset 17
model = helper.make_model(graph, producer_name="TiledMMExample", opset_imports=[helper.make_opsetid("", 17)])

# 驗證模型
onnx.checker.check_model(model)
print("Subgraph (2) for tiled matrix multiplication created successfully.")

# 保存子圖為 ONNX 模型
onnx.save(model, "./tiled_mm_subgraph.onnx")


Subgraph (2) for tiled matrix multiplication created successfully.


# 2-3-3. replace the Linear layers in the AlexNet with the equivalent subgraphs (2)
- saved the transformed model graph for HW2-4

In [11]:
# import onnx
# import numpy as np
# from onnx import numpy_helper

# # 加载 ONNX 模型
# model_path = "./alexnet.onnx"
# model = onnx.load(model_path)

# # 提取 Gemm 层的参数
# gemm_nodes = [node for node in model.graph.node if node.op_type == "Gemm"]
# for gemm_node in gemm_nodes:
#     print("Gemm Node:", gemm_node.name)
#     print("Inputs:", gemm_node.input)
#     print("Outputs:", gemm_node.output)
#     print("Attributes:", gemm_node.attribute)

In [12]:
# import onnx
# from onnx import shape_inference
# from collections import OrderedDict

# def get_layer_shapes(model_path, input_shape=None):
#     # 加载模型并执行形状推理
#     model = onnx.load(model_path)
#     model = shape_inference.infer_shapes(model)  # 自动补全形状信息

#     # 提取所有张量的形状信息
#     tensor_shapes = OrderedDict()

#     # 1. 提取模型的输入形状
#     for input_tensor in model.graph.input:
#         shape = [dim.dim_value if dim.dim_value > 0 else -1  # -1 表示动态维度
#                for dim in input_tensor.type.tensor_type.shape.dim]
#         tensor_shapes[input_tensor.name] = shape

#     # 2. 提取中间层的形状（value_info）
#     for value_info in model.graph.value_info:
#         shape = [dim.dim_value if dim.dim_value > 0 else -1
#                 for dim in value_info.type.tensor_type.shape.dim]
#         tensor_shapes[value_info.name] = shape

#     # 3. 提取输出形状
#     for output_tensor in model.graph.output:
#         shape = [dim.dim_value if dim.dim_value > 0 else -1
#                 for dim in output_tensor.type.tensor_type.shape.dim]
#         tensor_shapes[output_tensor.name] = shape

#     # # 4. 遍历所有节点，生成输入/输出形状映射
#     # layer_shapes = OrderedDict()

#     # # 初始输入形状（如果用户提供了输入形状）
#     # if input_shape is not None:
#     #     input_name = model.graph.input[0].name
#     #     tensor_shapes[input_name] = input_shape

#     # for node in model.graph.node:
#     #     # 提取当前节点的输入/输出名称
#     #     node_inputs = node.input
#     #     node_outputs = node.output

#     #     # 生成当前层的输入形状
#     #     input_shapes = []
#     #     for input_name in node_inputs:
#     #         if input_name in tensor_shapes:
#     #             input_shapes.append(tensor_shapes[input_name])
#     #         else:
#     #             input_shapes.append(None)  # 未知形状

#     #     # 生成当前层的输出形状
#     #     output_shapes = []
#     #     for output_name in node_outputs:
#     #         if output_name in tensor_shapes:
#     #             output_shapes.append(tensor_shapes[output_name])
#     #         else:
#     #             output_shapes.append(None)

#     #     # 将结果保存为 {层名称: {inputs: [...], outputs: [...]}}
#     #     layer_name = f"/{node.name}"  # 用节点名称作为键
#     #     layer_shapes[layer_name] = {
#     #         "inputs": {name: shape for name, shape in zip(node_inputs, input_shapes)},
#     #         "outputs": {name: shape for name, shape in zip(node_outputs, output_shapes)}
#     #     }

#     # return layer_shapes, tensor_shapes
#     return tensor_shapes

# # 使用示例
# model_path = "./alexnet.onnx"
# # layer_shapes, tensor_shapes = get_layer_shapes(model_path, input_shape=[1, 3, 224, 224])
# tensor_shapes = get_layer_shapes(model_path, input_shape=[1, 3, 224, 224])

# # # 转换为用户要求的格式：{"层名称/输出名称": 形状}
# # result = {}
# # for layer_name, shapes in layer_shapes.items():
# #     for output_name, shape in shapes["outputs"].items():
# #         key = f"{layer_name}/{output_name}"  # 例如 "/Conv_1/Conv_output_0"
# #         result[key] = shape

# # 添加输入形状
# # input_name = model.graph.input[0].name
# # result["actual_input_1"] = tensor_shapes[input_name]


In [13]:
# for tensor in tensor_shapes:
#     print(tensor,tensor_shapes[tensor])

In [14]:
import numpy as np
import onnx
from onnx import helper, numpy_helper, shape_inference
from onnx import AttributeProto, TensorProto
from collections import OrderedDict

def split_blocks(total_size, block_size):
    if total_size <= 0:
        return []
    num_blocks = total_size // block_size
    remainder = total_size % block_size
    return [block_size] * num_blocks + ([remainder] if remainder != 0 else [])

def validate_node_connections(model):
    existing_outputs = set()
    existing_initializers = set(init.name for init in model.graph.initializer)
    existing_inputs = set(input.name for input in model.graph.input)
    
    # 收集所有節點的輸出
    for node in model.graph.node:
        for output in node.output:
            existing_outputs.add(output)
    
    # 驗證每個輸入是否有效
    for node in model.graph.node:
        for input_name in node.input:
            if (input_name not in existing_outputs and 
                input_name not in existing_initializers and 
                input_name not in existing_inputs):
                raise ValueError(f"未定義的輸入: {input_name} 在節點 {node.name}")

def replace_gemm_with_blocks(model, max_block_size=1024):
    model = shape_inference.infer_shapes(model)
    value_info = {info.name: info for info in model.graph.value_info}
    initializers = {init.name: init for init in model.graph.initializer}
    
    new_nodes = []
    new_initializers = list(model.graph.initializer)  # 保留所有原始初始器
    name_counter = 0
    
    for node in model.graph.node:
        if node.op_type != 'Gemm':
            new_nodes.append(node)
            continue
            
        # 提取 Gemm 參數
        transA, transB, alpha, beta = 0, 0, 1.0, 1.0
        for attr in node.attribute:
            if attr.name == 'transA': transA = attr.i
            if attr.name == 'transB': transB = attr.i
            if attr.name == 'alpha': alpha = attr.f
            if attr.name == 'beta': beta = attr.f
        
        A_input = node.input[0]
        B_input = node.input[1]
        C_input = node.input[2] if len(node.input) > 2 else None
        
        # 處理權重矩陣 B
        B = numpy_helper.to_array(initializers[B_input]).copy()
        if transB:
            B = B.T
        if alpha != 1.0:
            B *= alpha
        
        # 獲取實際運算維度
        A_shape = [d.dim_value for d in value_info[A_input].type.tensor_type.shape.dim]
        if transA:
            M = A_shape[0]  # 轉置後的行數
        else:
            M = A_shape[1]
        N = B.shape[1]
        
        # 動態分塊
        block_sizes_M = split_blocks(M, max_block_size)
        block_sizes_N = split_blocks(N, max_block_size)
        
        # ================= 構建分塊權重 =================
        B_blocks = []
        for i, block_m in enumerate(block_sizes_M):
            row_start = sum(block_sizes_M[:i])
            row_end = row_start + block_m
            for j, block_n in enumerate(block_sizes_N):
                col_start = sum(block_sizes_N[:j])
                col_end = col_start + block_n
                block = B[row_start:row_end, col_start:col_end]
                
                # 生成唯一名稱
                block_name = f"{B_input}_block_{i}_{j}_{name_counter}"
                name_counter += 1
                B_blocks.append((i, j, block_name))
                new_initializers.append(
                    numpy_helper.from_array(block.astype(np.float32), name=block_name)
                )
        
        # ================= 重構計算圖 =================
        # 步驟1：拆分輸入A
        split_axis = 0 if transA else 1  # 根據transA選擇拆分軸
        split_outputs = [f"{node.name}_A_block_{i}_{name_counter}" for i in range(len(block_sizes_M))]
        # 創建 split 參數的 initializer
        split_tensor_name = f"{node.name}_split_sizes_{name_counter}"
        split_initializer = numpy_helper.from_array(
            np.array(block_sizes_M, dtype=np.int64),
            name=split_tensor_name
        )
        new_initializers.append(split_initializer)
        
        split_node = helper.make_node(
            "Split",
            inputs=[A_input, split_tensor_name],  # ONNX opset>=13 使用輸入參數
            outputs=split_outputs,
            axis=split_axis
        )
        new_nodes.append(split_node)
        name_counter += 1
        
        # 步驟2：矩陣乘法分塊
        matmul_outputs = []
        for (i, j, B_block_name) in B_blocks:
            A_block_name = split_outputs[i]
            
            # 處理transA
            if transA:
                transpose_node = helper.make_node(
                    "Transpose",
                    inputs=[A_block_name],
                    outputs=[f"{A_block_name}_transposed"],
                    perm=[1, 0]
                )
                new_nodes.append(transpose_node)
                A_block_name = f"{A_block_name}_transposed"
            
            matmul_name = f"{node.name}_MatMul_{i}_{j}_{name_counter}"
            name_counter += 1
            matmul_output = f"{matmul_name}_output"
            matmul_node = helper.make_node(
                "MatMul",
                inputs=[A_block_name, B_block_name],
                outputs=[matmul_output]
            )
            new_nodes.append(matmul_node)
            matmul_outputs.append((j, matmul_output))
        
        # 步驟3：按列求和
        sum_outputs = []
        for j in range(len(block_sizes_N)):
            sum_inputs = [out for idx, out in matmul_outputs if idx == j]
            if not sum_inputs:
                continue
            
            if len(sum_inputs) == 1:
                sum_output = sum_inputs[0]
            else:
                sum_name = f"{node.name}_Sum_{j}_{name_counter}"
                name_counter += 1
                sum_node = helper.make_node(
                    "Sum",
                    inputs=sum_inputs,
                    outputs=[sum_name]
                )
                new_nodes.append(sum_node)
                sum_output = sum_name
            sum_outputs.append(sum_output)
        
        # 步驟4：拼接結果
        concat_node = helper.make_node(
            "Concat",
            inputs=sum_outputs,
            outputs=[f"{node.name}_Concat_{name_counter}"],
            axis=1
        )
        name_counter += 1
        new_nodes.append(concat_node)
        final_output = f"{node.name}_Concat_{name_counter-1}"
        
        # 步驟5：處理偏置
        if C_input is not None:
            if beta != 1.0:
                # 添加beta縮放
                beta_array = np.array([beta], dtype=np.float32)
                beta_initializer = numpy_helper.from_array(beta_array, name=f"{node.name}_beta_{name_counter}")
                new_initializers.append(beta_initializer)
                beta_node = helper.make_node(
                    "Mul",
                    inputs=[C_input, beta_initializer.name],
                    outputs=[f"{node.name}_ScaledBias_{name_counter}"]
                )
                new_nodes.append(beta_node)
                C_input = f"{node.name}_ScaledBias_{name_counter}"
                name_counter += 1
            
            add_node = helper.make_node(
                "Add",
                inputs=[final_output, C_input],
                outputs=node.output
            )
            new_nodes.append(add_node)
        else:
            identity_node = helper.make_node(
                "Identity",
                inputs=[final_output],
                outputs=node.output
            )
            new_nodes.append(identity_node)
    
    # ================= 更新模型 =================
    model.graph.ClearField('node')
    model.graph.node.extend(new_nodes)
    
    model.graph.ClearField('initializer')
    model.graph.initializer.extend(new_initializers)
    
    # 形狀推斷與驗證
    model = shape_inference.infer_shapes(model)
    validate_node_connections(model)
    
    return model

def validate_model_transformation(orig_model_path, modified_model_path):
    import onnxruntime as ort
    
    # 加載原始模型
    orig_model = onnx.load(orig_model_path)
    orig_input_name = orig_model.graph.input[0].name
    
    # 生成測試數據
    np.random.seed(0)
    input_shape = [dim.dim_value for dim in orig_model.graph.input[0].type.tensor_type.shape.dim]
    input_data = np.random.rand(*input_shape).astype(np.float32)
    
    # 原始模型推理
    sess_orig = ort.InferenceSession(orig_model_path)
    output_orig = sess_orig.run(None, {orig_input_name: input_data})[0]
    
    
    
    sess_mod = ort.InferenceSession(modified_model_path)
    mod_input_name = sess_mod.get_inputs()[0].name
    output_mod = sess_mod.run(None, {mod_input_name: input_data})[0]
    
    # 數值驗證
    try:
        np.testing.assert_allclose(output_orig, output_mod, atol=1e-5)
        print("✅ 驗證成功！輸出一致")
    except AssertionError as e:
        max_diff = np.max(np.abs(output_orig - output_mod))
        print(f"⚠️ 驗證失敗！最大絕對誤差: {max_diff:.2e}")
        # raise

if __name__ == "__main__":
    # 使用範例
    orig_model_path = "./alexnet.onnx"
    modified_model_path = "./alexnet_modified.onnx"
    # 修改模型推理
    orig_model = onnx.load(orig_model_path)
    modified_model = replace_gemm_with_blocks(orig_model)
    modified_model = shape_inference.infer_shapes(modified_model)

    # 验证修改后的模型
    onnx.checker.check_model(modified_model)
    print("模型验证通过！")

    onnx.save(modified_model, modified_model_path)
    print("成功創建新的模型！")

模型验证通过！
成功創建新的模型！


# 2-3-4. Correctness Verification

In [15]:
import numpy as np
import onnxruntime as ort
import time

def validate_models(original_model_path, transformed_model_path, num_samples=100):
    # 初始化推理會話
    orig_sess = ort.InferenceSession(original_model_path)
    trans_sess = ort.InferenceSession(transformed_model_path)

    # 獲取模型輸入/輸出信息
    orig_input_name = orig_sess.get_inputs()[0].name
    trans_input_name = trans_sess.get_inputs()[0].name
    output_name = orig_sess.get_outputs()[0].name  # 假設單一輸出

    # 獲取輸入形狀 (支持動態 batch_size)
    input_shape = list(orig_sess.get_inputs()[0].shape)
    input_shape = [dim if dim > 0 else 1 for dim in input_shape]  # 處理動態維度

    # 初始化驗證結果
    all_passed = True
    failed_samples = []

    # 執行批量驗證
    print(f"開始驗證 {num_samples} 個隨機樣本...")
    start_time = time.time()
    
    for i in range(num_samples):
        # 生成隨機輸入數據
        np.random.seed(i)  # 為可重現性設置種子
        input_data = np.random.rand(*input_shape).astype(np.float32)
        
        # 原始模型推理
        orig_output = orig_sess.run([output_name], {orig_input_name: input_data})[0]
        
        # 轉換模型推理
        trans_output = trans_sess.run([output_name], {trans_input_name: input_data})[0]
        
        # 比較結果
        if not np.allclose(orig_output, trans_output, atol=1e-5):
            max_diff = np.max(np.abs(orig_output - trans_output))
            print(f"樣本 {i} 驗證失敗，最大差異: {max_diff:.2e}")
            all_passed = False
            failed_samples.append(i)
            
        # 進度顯示
        if (i+1) % 10 == 0:
            elapsed = time.time() - start_time
            print(f"已處理 {i+1}/{num_samples} 個樣本，耗時 {elapsed:.2f} 秒， 目前成功數為 {i+1 - len(failed_samples)}")

    # 最終報告
    total_time = time.time() - start_time
    print("\n驗證結果:")
    print(f"總樣本數: {num_samples}")
    print(f"通過樣本數: {num_samples - len(failed_samples)}")
    print(f"失敗樣本數: {len(failed_samples)}")
    print(f"總耗時: {total_time:.2f} 秒")
    
    if all_passed:
        print("✅ 所有樣本驗證成功！")
    else:
        print(f"❌ 發現 {len(failed_samples)} 個失敗樣本:")
        print(f"失敗樣本索引: {failed_samples}")
        raise ValueError("模型輸出不匹配")

if __name__ == "__main__":
    # 模型路徑配置
    original_model = "./alexnet.onnx"
    transformed_model = "./alexnet_modified.onnx"

    # 執行驗證
    validate_models(original_model, transformed_model, num_samples=100)

2025-03-08 09:09:20.946614042 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'learned_12'. It is not used by any node and should be removed from the model.
2025-03-08 09:09:20.947065292 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'learned_14'. It is not used by any node and should be removed from the model.
2025-03-08 09:09:20.947077625 [W:onnxruntime:, graph.cc:4198 CleanUnusedInitializersAndNodeArgs] Removing initializer 'learned_10'. It is not used by any node and should be removed from the model.


開始驗證 100 個隨機樣本...
已處理 10/100 個樣本，耗時 6.41 秒， 目前成功數為 10
已處理 20/100 個樣本，耗時 12.54 秒， 目前成功數為 20
已處理 30/100 個樣本，耗時 18.68 秒， 目前成功數為 30
已處理 40/100 個樣本，耗時 26.76 秒， 目前成功數為 40
已處理 50/100 個樣本，耗時 35.10 秒， 目前成功數為 50
已處理 60/100 個樣本，耗時 43.37 秒， 目前成功數為 60
已處理 70/100 個樣本，耗時 51.50 秒， 目前成功數為 70
已處理 80/100 個樣本，耗時 59.75 秒， 目前成功數為 80
已處理 90/100 個樣本，耗時 67.74 秒， 目前成功數為 90
已處理 100/100 個樣本，耗時 76.06 秒， 目前成功數為 100

驗證結果:
總樣本數: 100
通過樣本數: 100
失敗樣本數: 0
總耗時: 76.06 秒
✅ 所有樣本驗證成功！
